In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd

In [ ]:
import sys
sys.path.append(r'../')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
proj_name = 'output_FJSJ_09-02/'

In [ ]:
filename_basic = proj_name+'Basic_info.npy'
info_basic = np.load(filename_basic, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
key_subworks = ['01-01']
key_subworks = info_basic['key_subworks']
#key_subworks.remove('2013_10')
key_subworks

### read all stations coordinates

In [ ]:
info_basic['key_subworks']

In [ ]:
d_len = info_basic['d_len']

In [ ]:
stalistname = info_basic['stalistname']
stainfo = pd.read_excel(stalistname+'.xlsx')
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [ ]:
def FJ2TJ(f,ds_linear,flag_inter):
    T = 1./f[1:]
    dt = flag_inter*(T[-2]-T[-1])
    T1 = np.arange(T[-2],T[1],dt)
    f1 = np.sort(1./T1[::flag_inter])

    ds_linear1 = np.zeros((np.shape(ds_linear)[0],np.shape(f1)[0]))
    for i in range(np.shape(ds_linear)[0]):
        f_inter = interpolate.interp1d(f,ds_linear[i,:],kind='linear')
        ds_linear1[i,:] = f_inter(f1)
    indx = np.argsort(1./f1)
    T1 = 1./f1[indx]
    ds_linear2 = ds_linear1[:,indx]
    return T1,ds_linear2

In [ ]:
def plot_fj(ax,ds_linear,title0,f,c,xlim,index,cl):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=0.05, cmap = 'jet',clim=cl)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

In [ ]:
for key_subwork in key_subworks[0:1]:
    fig,ax = plt.subplots(2,2,figsize=(15,12))
    # settings for subwork
    dir_stack = info_basic['dir_stack']
    dir_ds = info_basic['dir_ds']
    #d_len = info_basic['d_len']
    dir_image = info_basic['dir_image']
    
    start = info_basic['start'][key_subwork]
    interval = info_basic['interval'][key_subwork]
    flag_time = info_basic['flag_time'][key_subwork]

    # plot area
    stalistname = proj_name+info_basic['stalistname'] +'-'+key_subwork
    stainfo = pd.read_excel(stalistname+'.xlsx')
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)

    # plot ncfs and ncfst

    outname = key_subwork+'_gather.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs_linear'][:]

    #outname = key_subwork+'_gather_linear.h5'
    #ncffile = h5py.File(dir_stack + outname,'r')
    #ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    f = info_basic['f']
    t = info_basic['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    title0 = "Linear Stack subarea time for "+key_subwork
    title1 = "Linear Stack subarea frequency for "+key_subwork
    xlim_f = [0,25]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title0,xlim_f,0)
    
    # plot fj
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    c = info_basic['c']
    #c = np.linspace(100,2000,1000)
    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    clim = [0,1]
    ax[1][0] = plot_fj(ax[1][0],ds_linear,title0,f,c,xlim_f,0,clim)
    ax[1][0].set_xlim(xlim_f)

    # plot FJ period
    #flag_inter = 10
    #T1,ds_linear2 = FJ2TJ(f,ds_linear,flag_inter)
    #xlim_T = [0.04,0.2]
    #title0 = "Linear stack period dispersion curve "+str(d_len)+' days'
    #indx = np.where((T1>=xlim_T[0]) & (T1<=xlim_T[1]))
    #ax[1][1] = plotlib.plot_fj_T(ax[1][1],ds_linear2[:,0:np.max(indx)],title0,T1[0:np.max(indx)],c,xlim_T,0)
    #ax[1][1].set_xlim(xlim_T)
    xlim_T = [-1,1]
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_linear[start::interval],r[start::interval],title1,flag_time,xlim_T,0)


    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_standard-output.png',dpi=60)

In [ ]:
%%capture
for key_subwork in key_subworks:
    fig,ax = plt.subplots(2,2,figsize=(15,12))
    # settings for subwork
    dir_stack = info_basic['dir_stack']
    dir_ds = info_basic['dir_ds']
    #d_len = info_basic['d_len']
    dir_image = info_basic['dir_image']
    
    start = info_basic['start'][key_subwork]
    interval = info_basic['interval'][key_subwork]
    flag_time = info_basic['flag_time'][key_subwork]

    # plot area
    stalistname = proj_name+info_basic['stalistname'] +'-'+key_subwork
    stainfo = pd.read_excel(stalistname+'.xlsx')
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)

    # plot ncfs and ncfst

    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]

    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    f = info_basic['f']
    t = info_basic['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    title0 = "Linear Stack subarea time for "+key_subwork
    title1 = "Linear Stack subarea frequency for "+key_subwork
    #xlim_f = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title0,xlim_f,0)
    
    # plot fj
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    c = info_basic['c']
    #c = np.linspace(100,2000,1000)
    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[1][0] = plotlib.plot_fj(ax[1][0],ds_linear,title0,f,c,xlim_f,0)
    ax[1][0].set_xlim(xlim_f)

    # plot FJ period
    #flag_inter = 10
    #T1,ds_linear2 = FJ2TJ(f,ds_linear,flag_inter)
    #xlim_T = [0.04,0.2]
    #title0 = "Linear stack period dispersion curve "+str(d_len)+' days'
    #indx = np.where((T1>=xlim_T[0]) & (T1<=xlim_T[1]))
    #ax[1][1] = plotlib.plot_fj_T(ax[1][1],ds_linear2[:,0:np.max(indx)],title0,T1[0:np.max(indx)],c,xlim_T,0)
    #ax[1][1].set_xlim(xlim_T)
    #xlim_T = [-1,1]
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_linear[start::interval],r[start::interval],title1,flag_time,xlim_T,0)


    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_standard-output.png',dpi=100)

In [ ]:
"""
%%capture
for key_subwork in key_subworks:
    fig,ax = plt.subplots(2,2,figsize=(15,12))
    # settings for subwork
    dir_stack = info_basic['dir_stack']
    dir_ds = info_basic['dir_ds']
    #d_len = info_basic['d_len']
    dir_image = info_basic['dir_image']
    
    start = info_basic['start'][key_subwork]
    interval = info_basic['interval'][key_subwork]
    flag_time = info_basic['flag_time'][key_subwork]

    # plot area
    stalistname = proj_name+info_basic['stalistname'] +'-'+key_subwork
    stainfo = pd.read_excel(stalistname+'.xlsx')
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)

    # plot ncfs and ncfst

    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]

    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    f = info_basic['f']
    t = info_basic['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    title0 = "Linear Stack subarea time for "+key_subwork
    title1 = "Linear Stack subarea frequency for "+key_subwork
    xlim = [0,40]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title0,xlim,0)
    xlim = [-3,3]
    ax[1][0] = plotlib.plot_ncfst(ax[1][0],t, ncfst_linear[start::interval], r[start::interval], title1,flag_time,xlim,0)

    # plot fj
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    c = info_basic['c']
    #c = np.linspace(100,2000,1000)
    xlim = [0,40]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[1][1] = plotlib.plot_fj(ax[1][1],ds_linear,title0,f,c,xlim,0)
    ax[1][1].set_xlim(xlim)
    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_standard-output.png',dpi=60)
"""

In [ ]:
"""
%%capture
for key_subwork in key_subworks:
    fig,ax = plt.subplots(2,2,figsize=(15,12))
    # settings for subwork
    dir_stack = info_basic['dir_stack']
    dir_ds = info_basic['dir_ds']
    #d_len = info_basic['d_len']
    dir_image = info_basic['dir_image']
    
    start = info_basic['start'][key_subwork]
    interval = info_basic['interval'][key_subwork]
    flag_time = info_basic['flag_time'][key_subwork]

    # plot area
    stalistname = proj_name+info_basic['stalistname'] +'-'+key_subwork
    stainfo = pd.read_excel(stalistname+'.xlsx')
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)

    # plot ncfs and ncfst
    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    f = info_basic['f']
    t = info_basic['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    title0 = "Linear Stack subarea time for "+key_subwork
    title1 = "Linear Stack subarea frequency for "+key_subwork
    xlim = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title0,xlim,0)
    xlim = [-3,3]

    # plot fj
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    c = info_basic['c']
    #c = np.linspace(100,2000,1000)

    xlim = [0,15]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days, low frequency'
    ax[1][0] = plotlib.plot_fj(ax[1][0],ds_linear,title0,f,c,xlim,0)
    ax[1][0].set_xlim(xlim)

    xlim = [0,40]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[1][1] = plotlib.plot_fj(ax[1][1],ds_linear,title0,f,c,xlim,0)
    ax[1][1].set_xlim(xlim)
    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_2_standard-output.png',dpi=60)
"""

In [ ]:
"""
flag_inter = 5
T1,ds_linear2 = FJ2TJ(f,ds_linear,flag_inter)
xlim = [0,1.5]
title0 = "Linear stack dispersion curve "+str(d_len)+' days'
indx = np.where((T1>=xlim[0]) & (T1<=xlim[1]))
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax = plotlib.plot_fj_T(ax,ds_linear2[:,0:np.max(indx)],title0,T1[0:np.max(indx)],c,xlim,0)
ax.set_xlim(xlim)
plt.tight_layout()
"""